<a href="https://colab.research.google.com/github/meliksahb/Deep-RL/blob/main/DeepRL_cartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.12.0 keras-rl2==1.0.5 gym==0.25.2 pyglet

In [9]:
import gym
import random
import tensorflow as tf
import numpy as np

In [10]:
# this is just a task to balancing a stick on the pole

# crating the env
env = gym.make("CartPole-v1", render_mode="human") # render mode selected as human to see whats happening

episodes = 10

for episode in range(1, episodes+1):
    state = env.reset()  # initial state
    done = False
    score = 0

    while not done: # If you set while True, you can see whole simulation

      action = random.choice([0, 1]) # random agent chooses randomly actions
      # 0 going left, 1 going right
      _, reward, done, _ = env.step(action)  # at this version of gym takes 4 values, recent ones takes 5
      score += reward
      env.render()  # render env each iteration

    print(f"Episode {episode}, Score: {score}")

env.close()


/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Episode 1, Score: 22.0
Episode 2, Score: 24.0
Episode 3, Score: 43.0
Episode 4, Score: 20.0
Episode 5, Score: 19.0
Episode 6, Score: 17.0
Episode 7, Score: 19.0
Episode 8, Score: 17.0
Episode 9, Score: 21.0
Episode 10, Score: 18.0


In [18]:
# now try with deep reinforcement learning

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam  # In this version of keras, this is Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import numpy as np
# Below solution for tf 2.18 but in this code we use
# tensorflow==2.12.0 keras-rl2==1.0.5 gym==0.25.2

# Solution for tf 2.18 version (if you use tf 2.18 you must add code for building model)
# for running rl in the colab you must change the util.py
# go to /usr/local/lib/python3.11/dist-packages/rl/util.py
# change this line from tensorflow.keras.models import model_from_config
# to this from tensorflow.keras.saving import deserialize_keras_object as model_from_config
# it must be look like this:
# from tensorflow.keras.saving import deserialize_keras_object as model_from_config
# from tensorflow.keras.models import Sequential, Model

In [19]:
# crating the env

env = gym.make("CartPole-v1")

states = env.observation_space.shape[0]  # how many states are there (take first dimension)
actions = env.action_space.n  # how many actions are there (you can get dynamically for each env)

model = Sequential()
model.add(Flatten(input_shape=(1, states)))
model.add(Dense(24, activation="relu")) # You can change the parameters (num. neurons, activation)
model.add(Dense(24, activation="relu"))
model.add(Dense(actions, activation="linear"))


agent = DQNAgent(
    model=model,
    memory=SequentialMemory(limit=50000, window_length=1),
    policy=BoltzmannQPolicy(),
    nb_actions=actions,
    nb_steps_warmup=10,
    target_model_update=0.01  # like learning rate
)

agent.compile(Adam(learning_rate=0.001), metrics=["mae"])  # train agent

agent.fit(env, nb_steps=10000, visualize=False, verbose=1)  # set visualize true if you want to see training

results = agent.test(env, nb_episodes=10, visualize=True)
print(np.mean(results.history["episode_reward"]))

env.close()


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.11/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.11/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Training for 100 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 28:05 - reward: 1.0000

/usr/local/lib/python3.11/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   10/10000 [..............................] - ETA: 3:17 - reward: 1.0000

/usr/local/lib/python3.11/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
/usr/local/lib/python3.11/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


...

   34/10000 [..............................] - ETA: 9:24 - reward: 1.0000 

/usr/local/lib/python3.11/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 26 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.11/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 27 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.11/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 28 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.11/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 29 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
/usr/local/lib/python3.11/dist-packages/rl/memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 30 + 1) instead
  batch

   99/10000 [..............................] - ETA: 5:36 - reward: 1.0000done, took 3.587 seconds
Testing for 10 episodes ...
Episode 1: reward: 9.000, steps: 9


/usr/local/lib/python3.11/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


Episode 2: reward: 9.000, steps: 9
Episode 3: reward: 10.000, steps: 10
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 9.000, steps: 9
Episode 6: reward: 9.000, steps: 9
Episode 7: reward: 9.000, steps: 9
Episode 8: reward: 9.000, steps: 9
Episode 9: reward: 10.000, steps: 10
Episode 10: reward: 9.000, steps: 9
9.3
